In [ ]:
import librosa
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import IPython.display as ipd
from sklearn import preprocessing 
import winsound
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset

Importing the data and conversion to spectrogram

In [ ]:
model_type = []
maneuvering_direction = []
fault = []
ID = []

for file in os.listdir('./Dataset-ArIES/drone_A/A/train/mic1'):
    labels = file.split('_')
    model_type.append(labels[0])
    maneuvering_direction.append(labels[1])
    fault.append(labels[2])
    ID.append(file)
for file in os.listdir('./Dataset-ArIES/drone_B/B/train/mic1'):
    labels = file.split('_')
    model_type.append(labels[0])
    maneuvering_direction.append(labels[1])
    fault.append(labels[2])
    ID.append(file)
for file in os.listdir('./Dataset-ArIES/drone_C/C/train/mic1'):
    labels = file.split('_')
    model_type.append(labels[0])
    maneuvering_direction.append(labels[1])
    fault.append(labels[2])
    ID.append(file)

In [ ]:
for file in os.listdir('./Dataset/drone_A/A/train/mic2'):
    labels = file.split('_')
    model_type.append(labels[0])
    maneuvering_direction.append(labels[1])
    fault.append(labels[2])
    ID.append(file)
for file in os.listdir('./Dataset/drone_B/B/train/mic2'):
    labels = file.split('_')
    model_type.append(labels[0])
    maneuvering_direction.append(labels[1])
    fault.append(labels[2])
    ID.append(file)
for file in os.listdir('./Dataset/drone_C/C/train/mic2'):
    labels = file.split('_')
    model_type.append(labels[0])
    maneuvering_direction.append(labels[1])
    fault.append(labels[2])
    ID.append(file)

In [ ]:
data = np.array([ID, model_type, maneuvering_direction, fault])
df = pd.DataFrame(data.T, columns=['ID', 'model_type', 'maneuvering_direction', 'fault']) 
df = pd.get_dummies(df, columns=['model_type', 'maneuvering_direction', 'fault'], dtype=int)
df.to_csv('train.csv', index = False)
df

In [ ]:
mic1 = []
mic2 = []
for file in os.listdir('./Dataset/drone_A/A/train/mic1'):
    filename = os.path.join('./Dataset/drone_A/A/train/mic1', file)
    audio_data, sample_r = librosa.load(filename)
    mic1.append(audio_data)
for file in os.listdir('./Dataset/drone_A/A/train/mic2'):
    filename = os.path.join('./Dataset/drone_A/A/train/mic2', file)
    audio_data, sample_r = librosa.load(filename)
    mic2.append(audio_data)

In [ ]:
for file in os.listdir('./Dataset/drone_B/B/train/mic1'):
    filename = os.path.join('./Dataset/drone_B/B/train/mic1', file)
    audio_data, sample_r = librosa.load(filename)
    mic1.append(audio_data)
for file in os.listdir('./Dataset/drone_B/B/train/mic2'):
    filename = os.path.join('./Dataset/drone_B/B/train/mic2', file)
    audio_data, sample_r = librosa.load(filename)
    mic2.append(audio_data)

In [ ]:
for file in os.listdir('./Dataset/drone_C/C/train/mic1'):
    filename = os.path.join('./Dataset/drone_C/C/train/mic1', file)
    audio_data, sample_r = librosa.load(filename)
    mic1.append(audio_data)
for file in os.listdir('./Dataset/drone_C/C/train/mic2'):
    filename = os.path.join('./Dataset/drone_C/C/train/mic2', file)
    audio_data, sample_r = librosa.load(filename)
    mic2.append(audio_data)
print("Data loaded")

In [ ]:
def compute_sg(audio_data, sr=22050):
    spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sr)
    # mfccs = librosa.feature.mfcc(y=audio_data, sr=sr, S=spectrogram, n_mfcc=30)
    # for i in range(len(spectrogram)):
    #     for j in range(len(spectrogram[i])):
    #           spectrogram[i][j] = spectrogram[i][j]/np.max(abs(spectrogram[i]))
    return spectrogram

In [ ]:
a, _ = librosa.load('./Dataset/drone_A/A/test/mic1/A_B_MF1_0_ConstructionSite_6_snr=10.926310539796413.wav')
librosa.display.specshow(librosa.stft(a))

In [ ]:
train_array = []
for i in tqdm(range(len(mic1))):
    train_array.append(compute_sg(mic1[i]))
for i in tqdm(range(len(mic1))):
    train_array.append(compute_sg(mic2[i]))

In [ ]:
train_array

In [ ]:
train_array = np.array(train_array)

In [ ]:
train_array.shape

In [ ]:
train_array_2d = train_array.reshape(len(train_array), train_array.shape[1]*train_array.shape[2])
np.savetxt('sg_train.csv', train_array_2d, delimiter=',')

Training

In [ ]:
train_array_2d = np.loadtxt('sg_train.csv', delimiter=',')

In [ ]:
train_array = train_array_2d.reshape(len(train_array_2d), 128, 22)

In [ ]:
# winsound.Beep(400, 100)

In [ ]:
training_tensor = torch.tensor(train_array, dtype=torch.float32)
training_tensor = training_tensor.unsqueeze(1)

In [ ]:
output_data = pd.read_csv('train.csv', header = None)

In [ ]:
output_data

In [ ]:
maneuvering_direction = output_data[[4, 5, 6, 7, 8, 9]].values[1:]
maneuvering_direction = maneuvering_direction.astype('int32')
fault = output_data[[10, 11, 12, 13, 14, 15, 16, 17,18]].values[1:]
fault = fault.astype('int32')
model_type = output_data[[1, 2, 3]].values[1:]
model_type = model_type.astype('int32')

In [ ]:
model_type

In [ ]:
y1 = torch.tensor(model_type, dtype=torch.float16).cuda()
y2 = torch.tensor(maneuvering_direction, dtype=torch.float16).cuda()
y3 = torch.tensor(fault, dtype=torch.float16).cuda()
Xtr = torch.tensor(training_tensor, dtype=torch.float32).cuda()

In [ ]:
dataset = TensorDataset(Xtr, y1, y2, y3)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [ ]:
# torch.cuda.manual_seed(42)
# torch.manual_seed(42)
# torch.backends.cudnn.deterministic = True

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(planes)
        self.softplus1 = nn.Softplus(beta=1, threshold=20)  
        self.softplus2 = nn.Softplus(beta=1, threshold=20)
        self.relu = nn.ReLU(inplace=True)
        

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = self.softplus1(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        shortcut = self.shortcut(x)
        out += shortcut
        out = self.softplus2(out)  
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.sp = nn.Softplus(inplace=True)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.sp(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        return out

class mtl(nn.Module):
    def __init__(self, num_classes1=3, num_classes2=6, num_classes3=9):
        super(mtl, self).__init__()
        self.resnet18 = ResNet(BasicBlock, [2, 2, 2, 2])
        self.fc1 = nn.Linear(512, num_classes1)
        self.fc2 = nn.Linear(512, num_classes2)
        self.fc3 = nn.Linear(512, num_classes3)
        self.sm1 = nn.Softmax(dim=1)#dim=1)
        self.sm2 = nn.Softmax(dim=1)#dim=1)
        self.sm3 = nn.Softmax(dim=1)#dim=1)

    def forward(self, x):
        out = self.resnet18(x)
        out1 = self.sm1(self.fc1(out))#, dim=1)
        out2 = self.sm2(self.fc2(out))#, dim=1)
        out3 = self.sm3(self.fc3(out))#, dim=1)
        return out1, out2, out3

model = mtl(num_classes1=3, num_classes2=6, num_classes3=9).cuda()

In [ ]:
model.load_state_dict(torch.load('nnweights.pth'))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.1)            

In [ ]:
num_epochs = 50
for epoch in tqdm(range(num_epochs)):
    for batch_idx, (data, target1, target2, target3) in enumerate(dataloader):
        data, target1, target2, target3 = data.cuda(), target1.cuda(), target2.cuda(), target3.cuda()
        outputs1, outputs2, outputs3 = model(data)

        loss1 = criterion(outputs1, target1)
        loss2 = criterion(outputs2, target2)
        loss3 = criterion(outputs3, target3)
        total_loss = loss1 + loss2 + loss3
    
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        # scheduler.step()

        if (batch_idx+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(dataloader)}], Total Loss: {total_loss.item():.4f}')
# winsound.Beep(440, 500)

Testing (I had to test in batches as it was showing memory error so I have used DataLoader here as well)

In [ ]:
model.eval()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
mic1_test = []
mic2_test = []
for file in os.listdir('./Dataset-ArIES/drone_A/A/test/mic1'):
    filename = os.path.join('./Dataset-ArIES/drone_A/A/test/mic1', file)
    audio_data, sample_r = librosa.load(filename)
    mic1_test.append(audio_data)
for file in os.listdir('./Dataset-ArIES/drone_A/A/test/mic2'):
    filename = os.path.join('./Dataset-ArIES/drone_A/A/test/mic2', file)
    audio_data, sample_r = librosa.load(filename)
    mic2_test.append(audio_data)

for file in os.listdir('./Dataset-ArIES/drone_B/B/test/mic1'):
    filename = os.path.join('./Dataset-ArIES/drone_B/B/test/mic1', file)
    y, _ = librosa.load(filename)
    mic1_test.append(y)
for file in os.listdir('./Dataset-ArIES/drone_B/B/test/mic2'):
    filename = os.path.join('./Dataset-ArIES/drone_B/B/test/mic2', file)
    y, _ = librosa.load(filename)
    mic2_test.append(y)

for file in os.listdir('./Dataset-ArIES/drone_C/C/test/mic1'):
    filename = os.path.join('./Dataset-ArIES/drone_C/C/test/mic1', file)
    y, _ = librosa.load(filename)
    mic1_test.append(y)
for file in os.listdir('./Dataset-ArIES/drone_C/C/test/mic2'):
    filename = os.path.join('./Dataset-ArIES/drone_C/C/test/mic2', file)
    y, _ = librosa.load(filename)
    mic2_test.append(y)

print('Data loaded')

In [4]:
sg_array_test = []
for i in tqdm(range(len(mic1_test))):
    sg_array_test.append(compute_sg(mic1_test[i]))

NameError: name 'mic1_test' is not defined

In [ ]:
for i in tqdm(range(len(mic2_test))):
    sg_array_test.append(compute_sg(mic2_test[i]))

In [ ]:
sg_array_test = np.array(sg_array_test)

In [ ]:
sg_array_test.shape

In [ ]:
sg_array_test_2d = sg_array_test.reshape(sg_array_test.shape[0], sg_array_test.shape[1]*sg_array_test.shape[2])

In [ ]:
np.savetxt('sg_test.csv', sg_array_test_2d, delimiter=',')

In [ ]:
sg_array_test_2d = np.loadtxt('sg_test.csv', delimiter=',')
sg_array_test = sg_array_test_2d.reshape(len(sg_array_test_2d), 128, 22)
X_test = torch.tensor(sg_array_test, dtype=torch.float32)
X_test = X_test.unsqueeze(1)

In [ ]:
X_test = X_test.cuda()

In [ ]:
dataloader1 = DataLoader(X_test, batch_size=64, shuffle=False)

In [ ]:
model_type_mapping = {0: 'A', 1: 'B', 2: 'C'}
maneuvering_direction_mapping = {0: 'B', 1: 'C', 2: 'CC', 3: 'F', 4: 'L', 5: 'R'}
fault_mapping = {0: 'MF1', 1: 'MF2', 2: 'MF3', 3: 'MF4', 4: 'N', 5: 'PC1', 6: 'PC2', 7: 'PC3', 8: 'PC4'}

model_type_pred = []
maneuvering_direction_pred = []
fault_pred = []

with torch.no_grad(), tqdm(total=len(dataloader1)) as progress_bar:
    for batch_idx, (data) in enumerate(dataloader1):
        output1, output2, output3 = model(data)
        
        m1_indices = output1.argmax(dim=1).cpu().numpy()
        m2_indices = output2.argmax(dim=1).cpu().numpy()
        m3_indices = output3.argmax(dim=1).cpu().numpy()

        model_type_pred.extend([model_type_mapping.get(idx) for idx in m1_indices])
        maneuvering_direction_pred.extend([maneuvering_direction_mapping.get(idx) for idx in m2_indices])
        fault_pred.extend([fault_mapping.get(idx) for idx in m3_indices])
        
        progress_bar.update(1)

model_type_pred = np.array(model_type_pred)
maneuvering_direction_pred = np.array(maneuvering_direction_pred)
fault_pred = np.array(fault_pred)

In [ ]:
data = np.array([model_type_pred, maneuvering_direction_pred, fault_pred])
df = pd.DataFrame(data.T, columns=['model_type', 'maneuvering_direction', 'fault']) 
df.to_csv('predwithoutid.csv', index = False)

In [ ]:
predwithoutid = pd.read_csv('predwithoutid.csv')

In [ ]:
predwithoutid.isnull().sum()

In [ ]:
ID = []
# Extract labels from file paths and store them in a list
for file in os.listdir('./Dataset-ArIES/drone_A/A/test/mic1'):
    ID.append(file +'_mic1')
for file in os.listdir('./Dataset-ArIES/drone_B/B/test/mic1'):
    ID.append(file + '_mic1')
for file in os.listdir('./Dataset-ArIES/drone_C/C/test/mic1'):
    ID.append(file + '_mic1')
for file in os.listdir('./Dataset-ArIES/drone_A/A/test/mic2'):
    ID.append(file + '_mic2')
for file in os.listdir('./Dataset-ArIES/drone_B/B/test/mic2'):
    ID.append(file + '_mic2')
for file in os.listdir('./Dataset-ArIES/drone_C/C/test/mic2'):
    ID.append(file + '_mic2')

In [ ]:
predwithoutid.insert(0, 'ID', ID)

In [ ]:
predwithoutid['ID']

In [ ]:
predwithoutid.to_csv('pred1.csv', index = False)